In [22]:
import tensorflow as tf
import keras as k
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random as r
import numpy as np
import cv2
import imghdr
import os
import datetime
import PIL
import io
import sklearn
import itertools

from keras import backend
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.applications import ResNet50V2, Xception, EfficientNetB3, EfficientNetB4, EfficientNetB5
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from collections import Counter
from PIL import ImageFile, Image, ImageOps
from sklearn.metrics import confusion_matrix
from tensorboard.plugins import projector

ImageFile.LOAD_TRUNCATED_IMAGES = True
print(f'Tensorflow version {tf.version.VERSION}')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)


Tensorflow version 2.5.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Data processing

In [23]:
PATH_TRAIN = 'C:/Users/crims/Tensorflow/PublisherIdetification/realpublishers/training/'
PATH_TEST = 'C:/Users/crims/Tensorflow/PublisherIdetification/realpublishers/testing/'

class DataGenerator:
    def __init__(self, val_split, height, width, batch_size, steps):
        self.val_split = val_split
        self.height = height
        self.width = width
        self.batch_size = batch_size
        self.steps = steps
        
    def train_generator(self):    
        train_generator = ImageDataGenerator(rescale=1./255, validation_split=self.val_split)
        print('Training folder:')
        self.train_data = train_generator.flow_from_directory(PATH_TRAIN, target_size=(self.height, self.width),
                                                     class_mode = 'categorical', batch_size=self.batch_size, 
                                                     subset = 'training')
        print('Validation folder:')
        self.valid_data = train_generator.flow_from_directory(PATH_TRAIN, target_size=(self.height, self.width),
                                                     class_mode = 'categorical', batch_size=self.batch_size, 
                                                     subset = 'validation')

    def test_generator(self):
        test_generator = ImageDataGenerator(rescale=1./255)
        print('Test folder:')
        self.test_data = test_generator.flow_from_directory(PATH_TEST, target_size=(self.width, self.height), 
                                                   class_mode=None, batch_size=1, shuffle=False)
        print()
        
    def generator_params(self):
        # Print info about the generated data
        self.class_num = self.train_data.num_classes
        self.class_counter = list(Counter(self.train_data.classes).values())
        self.class_counter_valid = list(Counter(self.valid_data.classes).values())
        self.class_names = list(self.train_data.class_indices)
        self.labels = self.train_data.labels
        print(f'Train data class name and num {dict(zip(self.class_names, self.class_counter))}')
        print(f'Valid data class name and num {dict(zip(self.class_names, self.class_counter_valid))}')
        print(f'Num files trained {self.batch_size * self.steps} and validated {(self.batch_size * self.steps * self.val_split):.0f} per epoch')
        print(f'Images resized to {self.height}x{self.width} trained avg {(self.train_data.n  // (self.batch_size * self.steps)):.1f} epochs' )
        print()
        
    def clean_data(self):
        # Remove nontype files from data folder (run once EVER for data)
        filenames = self.test_data.filenames
        n = 0
        while n < self.test_data.n:
            path = f'{PATH_TRAIN}{filenames[n]}'
            image = cv2.imread(path)
            img_type = imghdr.what(path)
            if img_type != "jpeg":
                print(f'Removing image from {path}')
                os.remove(path)
                n += 1
            else:
                n += 1
        print('All done!') 

## Tensorboard

In [24]:
class Utils:
    #Code from Tensorflow tutorial Tensorboard
    @staticmethod
    def plot_to_image(figure):
      # Save the plot to a PNG in memory.
      buf = io.BytesIO()
      plt.savefig(buf, format='png')
      plt.close(figure)
      buf.seek(0)
      # Convert PNG buffer to TF image
      image = tf.image.decode_png(buf.getvalue(), channels=4)
      # Add the batch dimension
      image = tf.expand_dims(image, 0)
      return image
    
    @staticmethod           
    def plot_confusion_matrix(cm, class_names):
        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        figure = plt.figure(figsize=(10, 10))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Greens, vmin=0, vmax=1)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    
    @staticmethod
    def get_img(img_path):
        #img = Image.open(StringIO(img_path))
        img = tf.io.read_file(img_path)
        # convert the compressed string to a 3D uint8 tensor
        img = tf.image.decode_jpeg(img, channels=3)
        # resize the image to the desired size for your model
        img = tf.image.resize(img, (200, 300))
        return img
    

## Model

In [25]:
class BuildModel(DataGenerator):
    def __init__(self, data_generator, network, pooling, optimizer, learn_rate, epochs):
        self.width = data_generator.width
        self.height = data_generator.height
        self.class_num = data_generator.class_num
        self.class_names = data_generator.class_names
        self.steps = data_generator.steps
        self.val_steps = data_generator.steps * data_generator.val_split
        self.batch_size = data_generator.batch_size
        self.train_data = data_generator.train_data
        self.valid_data = data_generator.valid_data
        self.test_data = data_generator.test_data
        
        self.network = network
        self.lr = learn_rate
        self.opt = optimizer
        self.pool = pooling
        self.epochs = epochs
        self.samples = samples
        self.publisher_names = list(self.train_data.class_indices.keys())[0:self.class_num]
            
    def compile_model(self):
        base = self.network(include_top=False, weights='imagenet', 
                     input_shape=(self.width, self.height, 3), pooling=self.pool)
        opt = self.opt(learning_rate=self.lr)  
        x = base.output
        #x = layers.Dense(512, activation= 'relu')(x)
        x = layers.Dense(self.class_num, activation='softmax')(x)
        self.model = tf.keras.Model(base.input, x)
        self.model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer=opt)
        
        self.embeddings = tf.keras.Model(inputs=self.model.inputs, 
                                         outputs=self.model.layers[-2].output)
    
    def run_model(self, summary, confusion_matrix, image_visual, projector): 
        if summary == True:
            self.model.summary()
            
        self.current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M")
        self.log_dir = os.path.join(f'tensorboard_logs/{self.network.__name__}{self.pool}_'
                                    f'{self.opt.__name__}lr{self.lr}_E{self.epochs}'
                                    f'B{self.batch_size}-{self.current_time}')
        if confusion_matrix == True:
            cm_callback = k.callbacks.LambdaCallback(on_epoch_end= self.log_confusion_matrix)
            
        if image_visual == True:
            image_callback = k.callbacks.LambdaCallback(on_epoch_end= self.log_images)
            
        if projector == True:
            projector_callback = k.callbacks.LambdaCallback(on_train_end=self.log_projection)
        
        scalar_callback = k.callbacks.TensorBoard(log_dir = self.log_dir, update_freq=100, 
                                                  histogram_freq=0, profile_batch=100,
                                                  write_graph=True, write_images=True)
        
        self.file_writer_cm = tf.summary.create_file_writer(f'{self.log_dir}/cm')
        self.file_writer_image = tf.summary.create_file_writer(f'{self.log_dir}/image')

        self.model.fit(self.train_data, validation_data= self.valid_data, 
                       callbacks=[scalar_callback, cm_callback, image_callback, projector_callback], 
                       epochs=self.epochs, steps_per_epoch=self.steps, 
                       validation_steps=self.val_steps * 2,
                       batch_size=self.batch_size, verbose=1)
        


In [ ]:
mod1.model.save(f'{self.network.__name__}_publisherid-{self.current_time}')

## Logger

In [ ]:
class Logger(BuildModel):
    def __init__(self, build_model):
        self.width = build_model.width
        self.height = build_model.height
        self.embeddings = build_model.embeddings
        self.test_data = build_model.test_data
        self.model = build_model.model
        self.class_names = build_model.class_names
        self.test_num = build_model.test_num
        self.test_labels =  [0,1,2,3] * int(build_model.test_num) 
        self.current_time = build_model.current_time
        self.network = build_model.network
        
    def log_projection(self, logs):
        sprite_width = 50
        sprite_height = 75
        datapoints = 500
        logdir = os.path.join('tensorboard_logs')
        if not os.path.exists(logdir):
            os.makedirs(logdir)

        # Generate embeddings
        images_pil = []
        images_embeddings = []
        labels = []
        for x in list(r.sample(range(0,999), datapoints)):
            img_path = self.test_data.filepaths[x]
            img_tf = Utils.get_img(img_path)

            # Save both tf image for prediction and PIL image for sprite
            img_pil = np.array(Image.open(img_path).resize((sprite_width, sprite_height)))
            img_embedding = self.embeddings(tf.expand_dims(img_tf, axis=0))
            images_embeddings.append(np.array(img_embedding[0]))
            images_pil.append(img_pil)

            # create and store labels
            label = img_path[77:-4] 
            labels.append(label)      
            with open(os.path.join(f'{logdir}/projector/', 'metadata.tsv'), 'w') as f: 
                 for label in labels:
                    f.write(f'{label}\n')

        one_square_size = int(np.ceil(np.sqrt(len(images_embeddings))))
        tile_width = sprite_width * one_square_size
        tile_height = sprite_height * one_square_size
        spriteimage = Image.new(mode='RGBA', size=(tile_width, tile_height), color=(0,0,0,0))

        for count, image in enumerate(images_pil):
            div, mod = divmod(count, one_square_size)
            h_loc = sprite_height * div
            w_loc = sprite_width * mod
            image = Image.fromarray(image)
            spriteimage.paste(image, (w_loc, h_loc, w_loc+sprite_width, h_loc+sprite_height))
        spriteimage.save(os.path.join(f'{logdir}/projector/', 'sprite.png'))
        
        feature_vector = tf.Variable(images_embeddings)
        checkpoint = tf.train.Checkpoint(embedding=feature_vector)
        checkpoint.save(os.path.join(f'{logdir}/projector/', 'embedding.ckpt'))

        # Set up config
        config = projector.ProjectorConfig()
        embedding = config.embeddings.add( )
        embedding.tensor_name = 'embedding/.ATTRIBUTES/VARIABLE_VALUE'
        embedding.metadata_path = 'projector/metadata.tsv'
        embedding.sprite.image_path = 'projector/sprite.png'
        embedding.sprite.single_image_dim.extend((sprite_width, sprite_height))
        projector.visualize_embeddings(logdir, config)
            
    def log_confusion_matrix(self, epoch, logs):  
        # Use the model to predict the values from the test_images.
        test_pred_raw = self.model.predict(self.test_data)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix using sklearn.metrics
        cm = sklearn.metrics.confusion_matrix(self.test_labels, test_pred)
        figure = Utils.plot_confusion_matrix(cm, class_names=self.class_names)
        cm_image = Utils.plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with self.file_writer_cm.as_default():
            tf.summary.image(f'{self.network.__name__}-{self.current_time}', cm_image, step=epoch)
            
    def log_images(self, epoch, logs):
        # Create pubs list
        pubs = list(r.sample(range(0, 249), self.samples)) 
        pubs.extend(r.sample(range(250, 499), self.samples))
        pubs.extend(r.sample(range(500, 749), self.samples))
        pubs.extend(r.sample(range(750, 999), self.samples))
        # Create image paths from test_data
        data = []
        for i in pubs:
            img = load_img(self.test_data.filepaths[i])
            img = img.resize((self.width, self.height)) # width x height
            img_arr = np.asarray(img)
            data.append(img_arr)
            
        # Data should be in (BATCH_SIZE, H, W, C) 
        assert np.size(np.shape(self.data)) == 4
        # Create a figure to contain the plot.
        figure = plt.figure(figsize=(10,10))
        num_images = np.shape(self.data)[0]
        size = int(np.ceil(np.sqrt(num_images)))

        for i in range(len(self.pubs)):
            # Start next subplot.
            plt.subplot(size, size, i + 1, title=self.class_names[self.test_labels[self.pubs[i]]])
            plt.xticks([])
            plt.yticks([])
            plt.grid(False)
            plt.imshow(self.data[i], cmap=plt.cm.binary)

        with self.file_writer_image.as_default():        
                tf.summary.image(f'{self.network.__name__}-{self.current_time}', 
                                 Tensorboard.plot_to_image(figure), max_outputs=len(self.pubs), step=epoch)

In [26]:
#Args: valid split, width, height, batch_size, steps
# batch_size = 32 for Xception and 12 for efficientB5, 16 for B4, 24 for B3
dat1 = DataGenerator(0.15, 300, 200, 12, 100)
dat1.train_generator()
dat1.test_generator()
dat1.generator_info()

# If the position of objects is important Avg pool if not Max Pooling 
# Args: obj, network, pooling, optimizer, learn_rate, epochs, samples
mod1 = BuildModel(dat1, EfficientNetB5, 'avg', Adam, 8e-5, 30, 4) #xception ~ 5e-4, efficient ~ 8e-5, Resnet ~ 5e-5
mod1.compile_model()
mod1.run_model(False, True, True)


Training folder:
Found 8830 images belonging to 4 classes.
Validation folder:
Found 1556 images belonging to 4 classes.
Test folder:
Found 1000 images belonging to 1 classes.

Train data class name and num {'IEEE': 1009, 'Macmillan': 3041, 'Springer Nature': 3465, 'Wolters Kluwer Health': 1315}
Valid data class name and num {'IEEE': 177, 'Macmillan': 536, 'Springer Nature': 611, 'Wolters Kluwer Health': 232}
Num files trained 1200 and validated 180 per epoch
Images resized to 300x200 trained avg 7.0 epochs

Epoch 1/30
100/100 [==============================] - 72s 547ms/step - loss: 1.0177 - acc: 0.6092 - val_loss: 1.3619 - val_acc: 0.3500
Epoch 2/30
100/100 [==============================] - 50s 497ms/step - loss: 0.7062 - acc: 0.7408 - val_loss: 1.3244 - val_acc: 0.3194
Epoch 3/30
100/100 [==============================] - 50s 497ms/step - loss: 0.5213 - acc: 0.8242 - val_loss: 1.1769 - val_acc: 0.4833
Epoch 4/30
100/100 [==============================] - 50s 497ms/step - loss: 0.450

C:\Users\crims\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: EfficientNetB5_publisherid-20210819-2131\assets


In [27]:
mod1.model.save(f'{mod1.network.__name__}_publisherid-{mod1.current_time}')

C:\Users\crims\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: EfficientNetB5_publisherid-20210819-2131\assets


## Predictor

In [20]:
class Predictor(BuildModel):
    def __init__(self, build_model):
        self.test_data = build_model.test_data
        self.valid_data = build_model.valid_data
        self.test_labels = build_model.test_labels
        self.width = build_model.width
        self.height = build_model.height
        self.model = build_model.model
        self.class_names = build_model.class_names
        self.batch_size = build_model.batch_size
             
    def img_predict(self):
        n = r.randint(0, self.test_data.n)
        filenames = self.valid_data.filenames
        path = f'{PATH_TRAIN}{filenames[n]}'
        pic = mpimg.imread(path)
        plt.axis('off')
        plt.imshow(pic)
        plt.show()

        img = tf.keras.preprocessing.image.load_img(path, target_size=(self.width, self.height))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_batch = np.expand_dims(img_array, axis=0)
        img_processed = tf.keras.applications.xception.preprocess_input(img_batch)

        prediction = self.model(img_processed, training=False)
        Top_index = np.argsort(np.max(prediction, axis=0))[-1]
        Second_index = np.argsort(np.max(prediction, axis=0))[-2]

        sort = np.sort(max(prediction))
        print(f'Prediction {self.class_names[Top_index]} with conf {round(sort[len(sort) - 1]*100)}%')
        print(f'2nd predict {self.class_names[Second_index]} with conf {round(sort[len(sort) - 2] * 100)}%')           
        print(f'Answer is {filenames[n][:]}')
        
    def batch_evaluate(self, num_batches):
        self.steps = num_batches
        self.model.evaluate(self.valid_data, batch_size=self.batch_size, steps=self.steps)
        
    def batch_predict(self, num_predicts):
        i = num_predicts
        test_pred_raw = self.model.predict(self.test_data)
        test_pred = np.argmax(test_pred_raw, axis=1)
        test_labels = self.test_labels
        acc = sum(1 for x,y in zip(test_pred[0:i], test_labels[0:i]) if x == y) / len(test_labels[0:i])
        print(f'Accuracy of predictions is {100 * acc:.1f}')        
        

In [28]:
pred1 = Predictor(mod1)
#pred1.img_predict()
pred1.batch_predict(1000)
#pred1.batch_evaluate(49)

Accuracy of predictions is 78.400
